# **Connecting to a remote mysql database from COLAB**
#### This notebook has examples of how to connect to remote mysql databases from google colab:
#### It will first connect to the Panos Ipeirotis Mysql Server which houses a number of sample databases. This server is actually running in the Googe Cloud. We only have read-only access to it, so we can't add databases or create and modify databases.
#### The second server is running in the Stern Cloud, and is also a mysql server.
####  Each team has an account on this server, so they can create and store tables.


## First, make sure all the required software is installed.
### We need to install the linux mysqlclient libraries on our linux system, and the  python mysqlclient libraries are installed
#### Unfortunately, we need to do this for every colab session.

In [ ]:
# install the required linux support needed by the python libraries
!sudo apt-get install python3-dev default-libmysqlclient-dev
# Next install the python mysql support
!pip install mysqlclient

In [ ]:
# We also need the sqlalchemy package for python
!pip3 install sqlalchemy

## Then , we should be able to use sql statements directly from our notebooks

In [ ]:
from sqlalchemy import create_engine


### We need to create the credentials to connect to our database, including the server name
#### Note: these are the same credentials you will use from mysqlworkbench to connect

In [ ]:
conn_string_imdb = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='student', 
    password='dwdstudent2015', 
    host = 'db.ipeirotis.org', 
    port=3306, 
    db='imdb',
    encoding = 'utf-8'
)
engine_imdb = create_engine(conn_string_imdb)


### next install the sql magic code python library .
### so we can type sql queries in a code cell

In [ ]:
!sudo pip install sql_magic

## Now, configure a connection to the google database server
### Note the connection name matches what we used in the create_engine call above.

In [ ]:
%config SQL.conn_name = 'engine_imdb'

### And to run an SQL command against the database- Just put %% in front of the command

In [ ]:
%%read_sql
# Let's see what databases are on this server
show databases

Query started at 10:15:32 PM UTC; Query executed in 0.02 m

,Database
0,information_schema
1,bike_sharing
2,citibike
3,citibike_fall2017
4,facebook
5,imdb
6,music
7,nypd_complaints
8,public
9,restaurants


### Lets look at some tables


In [ ]:
%%read_sql 
# What is the table definition for the actors table?
 describe actors

Query started at 10:15:36 PM UTC; Query executed in 0.00 m

,Field,Type,Null,Key,Default,Extra
0,id,int(11),NO,PRI,0,
1,first_name,varchar(100),YES,MUL,None,
2,last_name,varchar(100),YES,MUL,None,
3,gender,char(1),YES,,None,


### And issue a query ...

In [ ]:
%read_sql select count(*) from actors;

Query started at 10:15:40 PM UTC; Query executed in 0.03 m

,count(*)
0,817718


# Now Switch to the class server, bigdata.stern.nyu.edu
### We will be using this later in the term for projects
#### Each team has an account and their own database.

In [ ]:
conn_string_big = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='DealingS21GB10', 
    password='DealingS21GB10!!', 
    host = 'bigdata.stern.nyu.edu', 
    port=3306, 
    db='DealingS21GB10',
    encoding = 'utf-8'
)
# call this connection engine_big
engine_big = create_engine(conn_string_big)


In [ ]:
# Tell sql magic the new connection to use
%config SQL.conn_name = 'engine_big'

### At now we are connected to the new server, bigdata.stern.nyu.edu
#### Let's issue some queries


In [ ]:
%%read_sql
show databases

Query started at 10:16:04 PM UTC; Query executed in 0.03 m

,Database
0,information_schema
1,DealingF19
2,TMP
3,dealingF19
4,dealingF19GB1
5,dealingF19GB10
6,dealingF19GB2
7,dealingF19GB3
8,dealingF19GB4
9,dealingF19GB5


In [ ]:
%read_sql show tables

Query started at 10:16:11 PM UTC; Query executed in 0.01 m

,Tables_in_dealingF19GB10
0,311calls
1,NYPDCALLS
2,Stations
3,mytable
4,newtable
5,test
6,thetable


And let's try our first query. You will use the `fetchall` command, which returns all the results.

In [ ]:
import MySQLdb  as mdb

con = mdb.connect(host, username, password, database, charset='utf8', use_unicode=True);

cur = con.cursor()
cur.execute("SELECT * FROM actors LIMIT 100")
results = cur.fetchall()
cur.close()
con.close()

print(results)

Now, let's fetch the data line by line, to avoid having long execution times when fetching the results

In [ ]:
import MySQLdb as mdb

con = mdb.connect(host, username, password, database, charset='utf8', use_unicode=True);

# Returning all data at a time may not be feasible. We can fetch rows one by one.
cur = con.cursor()
cur.execute("SELECT * FROM actors LIMIT 10")
# We fetch the rows one by one using the fetchone() method. 
# The rowcount property gives the number of rows returned by the SQL statement.
for i in range(cur.rowcount):
    row = cur.fetchone()
    print(row)
    print(row[0], row[1], row[2])
cur.close()
con.close()

**The dictionary cursor:** There are multiple cursor types in the MySQLdb module. The default cursor returns the data in a tuple of tuples. When we use a dictionary cursor (`mdb.cursors.DictCursor`), the data is placed in a Python dictionary, which allows us to refer to the data by their column names.

In [ ]:
import MySQLdb as mdb

# The dictionary cursor
# There are multiple cursor types in the MySQLdb module. 
# The default cursor returns the data in a tuple of tuples. 
# When we use a dictionary cursor, the data is sent in a form of Python dictionaries. 
# This way we can refer to the data by their column names.

con = mdb.connect(host, username, password, database, charset='utf8', use_unicode=True);
    
sql_query = "SELECT * FROM actors LIMIT 10"

cur = con.cursor(mdb.cursors.DictCursor)
cur.execute(sql_query)
rows = cur.fetchall()
for row in rows:
    print(row)
    print(row["id"], row["first_name"], row["last_name"])
    
cur.close()
con.close()